In [1]:
from sql_engine import connect
import datetime
import pandas as pd

In [2]:
def check_diff_days(diff_day):
    today = datetime.datetime.today()
    diff_day = datetime.timedelta(days=diff_day)
    prev_day = today-diff_day
    interval = today-prev_day
    days = interval.days
    begin_signal = datetime.datetime(2021, 1, 16)
    if prev_day<begin_signal:
        return (today-begin_signal).days
    else:
        return days

In [84]:
def load_yx_return_data(diff_days_start, diff_days_end, yx_con):
    diff_days_start = check_diff_days(diff_days_start)
    diff_days_end = check_diff_days(diff_days_end)
    return_SQL = f'''
	SELECT
		CAST(
		t1.id AS CHAR ( 30 )) id,
		t1.trans_time,
		LEFT ( t1.trans_time, 10 ) trans_date,
		t1.trans_no SourceNo,
		t5.PaymentAccount,
		t4.merchant_id,
		CAST(
		t3.user_id AS CHAR ( 30 )) user_id,
		t3.union_id UnionNumber,
		t3.name name,
		t2.out_id activityId,
        t2.policy_name,
		t1.title,
		t1.body,
	IF
		( t1.amt_trans > 0, t1.amt_trans / 100, 0 ) amt_in,
	IF
		( t1.amt_trans < 0, t1.amt_trans / 100, 0 ) amt_out,
	IF
		( t1.amt_trans > 0, t1.amt_trans / 100, 0 ) +
	IF
		( t1.amt_trans < 0, t1.amt_trans / 100, 0 ) money,
		t4.from_trans_id,
		t4.bank_trans_id
	FROM
		oryx_account.account_trans_record t1
		INNER JOIN oryx_policy.policy t2 ON t1.policy_id = t2.policy_id
		INNER JOIN oryx_agent.agent_info t3 ON t1.customer_id = t3.user_id
		LEFT JOIN (select merchant_id,trans_id,from_trans_id,bank_trans_id from oryx_pay.pay
		GROUP BY merchant_id,trans_id,from_trans_id,bank_trans_id) t4 ON t1.trans_no = t4.trans_id 
		left join oryx_bi.imp_account_returned_reflect t5 on t5.merchant_id = t4.merchant_id and t5.PaymentAccount != ''
	WHERE
		t1.company_id IN ( 12, 979 ) 
		AND t2.recharge_enable = 1 
		and t2.state = 1
		AND t1.customer_type = 0 
		AND t1.account_type = 0 
		AND ( t1.type NOT IN ( 'pay', 'refund' ) OR t1.title = '余额申请单' ) 
		AND t1.title NOT IN ( '历史订单取消发货单余额退款', '车票审核补报上账', '历史订单余额退款', '车票审核报销上账' ) 
		AND DATEDIFF( NOW(), t1.trans_time )<={diff_days_start} AND DATEDIFF(NOW(), t1.trans_time)>={diff_days_end}
		AND t1.policy_id IS NOT NULL 
		AND ( t1.remark NOT LIKE '%%测试%%' OR t1.remark IS NULL ) 
		AND t3.union_id not in ('CM66381516','CM00006317','CM00007650')
		and t4.merchant_id is not null
		ORDER BY t1.trans_time,t1.trans_no
                                        '''
    return_data = pd.read_sql(return_SQL, yx_con)
    return return_data

In [85]:
def load_erp_return_data(diff_days_start, diff_days_end, erp_con):
    diff_days_start = check_diff_days(diff_days_start)
    diff_days_end = check_diff_days(diff_days_end)
    return_SQL = f'''SELECT
a.OrderNo,
a.PaymentAccount,
ISNULL( auditime, a.createTime ) auditime,
CONVERT(varchar(100), ISNULL( auditime, a.createTime ), 23) trans_date,
	IIF (
		( a.SourceNo IS NULL OR LEN( a.SourceNo ) = 0 ) 
		AND a.OrderNo LIKE 'SCBA%',
		a.OrderNo,
		a.SourceNo 
	) SourceNo,
	IIF (
		a.agentId= 'dab70acc-9b11-44e9-bc95-d6ad1e573336',
		'CM00000035',
		ISNULL(
			IIF (
				b.UnionNumber= 'CM88810886',
				'CM00020001',
				IIF (
					b.UnionNumber= 'CM00007497',
					'CM00007482',
					IIF (
						b.UnionNumber= 'CM00007595',
						'CM00007113',
						IIF ( b.UnionNumber= 'CM00006816', 'CM00006194', IIF ( b.UnionNumber= 'CM00007136', 'CM00000304', b.UnionNumber ) ) 
					) 
				) 
			),
			c.UnionNumber 
		) 
	) UnionNumber,
	d.zctype,d.zcname,
	isnull(c.realname,b.RealName) realname,
	InValue,
	- OutValue OutValue,
	InValue - OutValue money,
    Item
FROM
	SCLMERPDB_UE.dbo.DH_BalanceApply a
	LEFT JOIN SCLM_DRPDB.dbo.b_UserInfo b ON a.agentId= b.f_id
	LEFT JOIN SCLM_DRPDB.dbo.b_UnionUserInfo c ON b.UnionNumber= c.UnionNumber 
	AND isdelete = 0
	LEFT JOIN dbo.ZCLX d ON a.activityid = d.zc_id 
WHERE
	a.status = 1 
	AND Deleted = 0 
	AND datediff( dd, ISNULL( auditime, a.createTime ), getdate( ) ) <= { diff_days_start } 
	AND datediff( dd, ISNULL( auditime, a.createTime ), getdate( ) ) >= { diff_days_end }
	AND d.ActivityCWTypeNo in (1,2)
	AND agentname NOT LIKE '%测试%' 
	AND Item IN (
		'货物预存款',
		'客户余额互转',
		'余额清理',
		'政策余额充值',
		'政策余额转款',
		'三合一转款',
		'全品牌转出',
		'代理清算',
		'卡位收支',
		'全品牌转入',
		'全品牌转款',
		'订单补差额',
		'全品牌A转B',
		'销售订单撤销',
		'政策赠送',
		'退赠送金额',
		'保证金收支' 
	) 
	AND agentwechat NOT IN ( 'Power', 'manna24', '13076952970', 'zhulei666', 'test05' )
	and c.UnionNumber not in ('CM00006317','CM00007650')
 	and a.PaymentAccount is not null and len(a.PaymentAccount) != 0
	ORDER BY ISNULL( auditime, a.createTime ),SourceNo'''
    return_data = pd.read_sql(return_SQL, erp_con)
    return return_data

In [107]:
def contrast_return(diff_days_start, diff_days_end):
    yx_con, server = connect('yx')
    erp_con = connect('new_ERP')
    bi_con = connect('bidata')
    yx_return_data = load_yx_return_data(diff_days_start, diff_days_end, yx_con)
    erp_return_data = load_erp_return_data(diff_days_start, diff_days_end, erp_con)
    yx_return_data['id'] = yx_return_data['id'].str.strip()
    yx_con.dispose()
    server.close()
    erp_con.dispose()
    data = pd.merge(yx_return_data[yx_return_data.SourceNo.notnull()], erp_return_data[erp_return_data.SourceNo.notnull()], how='outer', on=['SourceNo', 'UnionNumber','trans_date','PaymentAccount'],suffixes=('_yx', '_erp'))
    data['err_type'] = None
    data['err_type'][data.id.isnull()] = 'ERP有YX没有'
    data['err_type'][data.OrderNo.isnull()] = 'YX有ERP没有'
    data['err_type'][(data['err_type'].isnull())&(((data['InValue']-data['amt_in']).abs()>1)|((data['OutValue']-data['amt_out']).abs()>1))] = '金额不一致'
    data['err_type'][(data['Item'].isin(['销售订单撤销', '政策赠送', '退赠送金额', '保证金收支']))&(data['err_type']=='ERP有YX没有')] = None
    data = data[data.err_type.notnull()]
    data.reset_index(drop=True,inplace=True)
    bi_con.execute(f"DELETE FROM InterMediate_yx_erp_return_account_diff WHERE DATEDIFF(DD, trans_date, GETDATE()) BETWEEN {diff_days_start} AND {diff_days_end}")
    todo = set(data['trans_date'].astype('str').to_list())
    if len(todo)>0:
        todo = "('"+"','".join(todo)+"')"
        bi_con.execute(f"DELETE FROM InterMediate_yx_erp_return_account_diff WHERE trans_date IN {todo}")
    if len(data)>0:
        data.to_sql("InterMediate_yx_erp_return_account_diff", bi_con, index=None, if_exists='append', method='multi', chunksize=100)
    return

In [108]:
data = contrast_return(365,0)

In [93]:
data[(data['PaymentAccount'] == '127911006810502')]

,id,trans_time,trans_date,SourceNo,PaymentAccount,merchant_id,user_id,UnionNumber,name,activityId,...,OrderNo,auditime,zctype,zcname,realname,InValue,OutValue,money_erp,Item,err_type
3,33660969,2021-01-16 14:01:53,2021-01-16,2021011614013508181500,127911006810502,006673,5107888,CM00001124,杨攀,f11a718e-1dfd-4567-bfaf-3727ffe4597e,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YX有ERP没有
9,33661225,2021-01-16 14:06:32,2021-01-16,2021011614061739458210,127911006810502,006673,5011968,CM00001496,郝丽丽,0b447d34-5096-4f44-9759-5eb1beddc1f6,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YX有ERP没有
19,33661652,2021-01-16 14:09:30,2021-01-16,2021011614090596156920,127911006810502,006673,5018643,CM00004270,康新心,0b447d34-5096-4f44-9759-5eb1beddc1f6,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YX有ERP没有
21,33661918,2021-01-16 14:10:25,2021-01-16,2021011614095536193535,127911006810502,006673,4985179,CM00000274,龚开秀,0b447d34-5096-4f44-9759-5eb1beddc1f6,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YX有ERP没有
28,33662524,2021-01-16 14:13:10,2021-01-16,2021011614124097764617,127911006810502,006673,5079625,CM00001115,鲁礼萍,77e64a3e-6830-4011-b641-2970161c6cd0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YX有ERP没有
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810,NaN,NaT,2021-03-23,2021032310092075644803,127911006810502,NaN,NaN,CM00001334,NaN,NaN,...,SCBA4840112889990042995,2021-03-23 10:09:20,其它,全品牌-2021开门红家电补差款,刘迁影,800.0,0.0,800.0,政策余额充值,ERP有YX没有
7813,NaN,NaT,2021-03-23,2021030209573691576673,127911006810502,NaN,NaN,CM00006735,NaN,NaN,...,SCBA5706118475044823954,2021-03-23 17:47:20,混拿回款,全品牌-2021开门红混拿,吴锦燕,22500.0,0.0,22500.0,政策余额充值,ERP有YX没有
7820,NaN,NaT,2021-03-26,2021032611444496810630,127911006810502,NaN,NaN,CM00000496,NaN,NaN,...,SCBA5340290411953913507,2021-03-26 11:44:45,其它,全品牌-2021开门红家电补差款,马虹,800.0,0.0,800.0,政策余额充值,ERP有YX没有
7828,NaN,NaT,2021-03-28,2021032808490146233719,127911006810502,NaN,NaN,CM22720142,NaN,NaN,...,SCBA5668591276177211687,2021-03-28 08:49:01,其它,全品牌-2021开门红家电补差款,廖桂心,800.0,0.0,800.0,政策余额充值,ERP有YX没有


In [98]:
todo = set(data['trans_date'].astype('str').tolist())

AttributeError: 'set' object has no attribute 'head'

In [104]:
to = "('"+"','".join(todo)+"')"
to

"('2021-02-05','2021-03-13','2021-03-28','2021-02-06','2021-03-22','2021-03-17','2021-01-25','2021-01-28','2021-02-22','2021-03-20','2021-02-19','2021-02-25','2021-02-01','2021-03-12','2021-03-29','2021-04-07','2021-01-24','2021-04-09','2021-02-24','2021-03-02','2021-03-21','2021-03-31','2021-01-17','2021-02-27','2021-04-05','2021-04-02','2021-02-20','2021-03-01','2021-01-21','2021-03-09','2021-03-25','2021-01-26','2021-02-08','2021-03-18','2021-01-19','2021-02-23','2021-03-24','2021-03-11','2021-02-28','2021-03-27','2021-03-30','2021-02-02','2021-04-06','2021-03-16','2021-03-23','2021-02-03','2021-03-26','2021-01-30','2021-03-05','2021-03-06','2021-03-10','2021-03-07','2021-03-03','2021-01-16','2021-01-22','2021-03-19','2021-04-01','2021-02-21','2021-01-18','2021-01-23','2021-03-04','2021-01-27','2021-01-31','2021-02-26','2021-01-20','2021-03-14','2021-02-04','2021-01-29','2021-03-15','2021-03-08')"

In [78]:
data[(data['PaymentAccount'] == '11163000000788603') & (data['trans_date'] == '2021-03-22')]

,id,trans_time,trans_date,SourceNo,PaymentAccount,merchant_id,user_id,UnionNumber,name,activityId,...,OrderNo,auditime,zctype,zcname,realname,InValue,OutValue,money_erp,Item,err_type
7808,NaN,NaT,2021-03-22,2021032215400272617119,11163000000788603,NaN,NaN,CM00000943,NaN,NaN,...,SCBA5334331430634077044,2021-03-22 15:40:02,其它,全品牌-2021开门红家电补差款,张春平,800.0,0.0,800.0,政策余额充值,ERP有YX没有


In [ ]:
data['PaymentAccount'] == ''

In [21]:
yx_con, server = connect('yx')
erp_con = connect('new_ERP')


In [22]:
yx_return_data = load_yx_return_data(1, 1, yx_con)

In [23]:
yx_return_data

,id,trans_time,trans_date,SourceNo,PaymentAccount,merchant_id,user_id,UnionNumber,name,activityId,title,body,amt_in,amt_out,money,from_trans_id,bank_trans_id
0,40502644,2021-04-09 00:02:38,2021-04-09,2021040900023225307078,11163000000788603,1576019011,4997576,CM00000247,陈琴,0b447d34-5096-4f44-9759-5eb1beddc1f6,账户充值,账户充值,20.0,0.0,20.0,4200000900202104091907878769,
1,40502704,2021-04-09 00:07:34,2021-04-09,2021040900072897572483,11163000000788603,1576019011,4997576,CM00000247,陈琴,0b447d34-5096-4f44-9759-5eb1beddc1f6,账户充值,账户充值,20.0,0.0,20.0,4200000888202104097843545091,
2,40503749,2021-04-09 01:58:03,2021-04-09,2021040901573089360139,18627764493@163.com,2019012663181079,5126470,CM00000753,王腊艳,f11a718e-1dfd-4567-bfaf-3727ffe4597e,账户充值,账户充值,150.0,0.0,150.0,2021040922001439871444704450,
3,40503753,2021-04-09 02:00:16,2021-04-09,2021040902000852753284,11163000000788603,1576019011,5126470,CM00000753,王腊艳,f11a718e-1dfd-4567-bfaf-3727ffe4597e,账户充值,账户充值,450.0,0.0,450.0,4200000930202104097000146114,
4,40503757,2021-04-09 02:02:12,2021-04-09,2021040902020636737947,11163000000788603,1576019011,5126470,CM00000753,王腊艳,f11a718e-1dfd-4567-bfaf-3727ffe4597e,账户充值,账户充值,180.0,0.0,180.0,4200000886202104091581577153,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,40577634,2021-04-09 23:24:42,2021-04-09,2021040923241599356630,18627764493@163.com,2019012663181079,5093277,CM00001469,贾秀琴,95b7cd83-df45-42b5-b064-21d9cf399b2e,账户充值,账户充值,6000.0,0.0,6000.0,2021040922001464101431138268,
241,40577646,2021-04-09 23:25:11,2021-04-09,2021040923250450765017,127911006810701,1576018601,5093277,CM00001469,贾秀琴,95b7cd83-df45-42b5-b064-21d9cf399b2e,账户充值,账户充值,4000.0,0.0,4000.0,4200000946202104098001579612,
242,40577686,2021-04-09 23:28:07,2021-04-09,2021040923275153040212,18627764493@163.com,2019012663181079,4997995,CM00000704,陈燕妮,f11a718e-1dfd-4567-bfaf-3727ffe4597e,账户充值,账户充值,450.0,0.0,450.0,2021040922001472831449973290,
243,40577966,2021-04-09 23:38:54,2021-04-09,2021040923381181183847,18627756472@163.com,2021002121653504,5111921,CM00003870,崔俊波,95b7cd83-df45-42b5-b064-21d9cf399b2e,账户充值,账户充值,12500.0,0.0,12500.0,2021040922001421361431566045,


In [15]:
erp_return_data = load_erp_return_data(1, 1, erp_con)

In [16]:
erp_return_data

,merchant_id,auditime,trans_time,SourceNo,UnionNumber,zctype,zcname,realname,InValue,OutValue,money
0,11163000000788603,2021-04-09 00:02:39,2021-04-09,2021040900023225307078,CM00000247,总代价赠送款,全品牌常规总代价,陈琴,20.0,0.0,20.0
1,11163000000788603,2021-04-09 00:07:34,2021-04-09,2021040900072897572483,CM00000247,总代价赠送款,全品牌常规总代价,陈琴,20.0,0.0,20.0
2,18627764493@163.com,2021-04-09 01:58:06,2021-04-09,2021040901573089360139,CM00000753,小样款,全品牌小样款,王腊艳,150.0,0.0,150.0
3,11163000000788603,2021-04-09 02:00:16,2021-04-09,2021040902000852753284,CM00000753,小样款,全品牌小样款,王腊艳,450.0,0.0,450.0
4,11163000000788603,2021-04-09 02:02:11,2021-04-09,2021040902020636737947,CM00000753,小样款,全品牌小样款,王腊艳,180.0,0.0,180.0
...,...,...,...,...,...,...,...,...,...,...,...
242,18627764493@163.com,2021-04-09 23:24:42,2021-04-09,2021040923241599356630,CM00001469,混拿回款,全品牌-2021开门红混拿,贾秀琴,6000.0,0.0,6000.0
243,127911006810701,2021-04-09 23:25:11,2021-04-09,2021040923250450765017,CM00001469,混拿回款,全品牌-2021开门红混拿,贾秀琴,4000.0,0.0,4000.0
244,18627764493@163.com,2021-04-09 23:28:08,2021-04-09,2021040923275153040212,CM00000704,小样款,全品牌小样款,陈燕妮,450.0,0.0,450.0
245,18627756472@163.com,2021-04-09 23:38:56,2021-04-09,2021040923381181183847,CM00003870,混拿回款,全品牌-2021开门红混拿,崔俊波,12500.0,0.0,12500.0
